<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# FIA Drought Model

_by Jeremy Freeman (CarbonPlan), October 26, 2020_

This notebook demos drought modelling.


In [1]:
import numpy as np
import pandas as pd
from carbonplan_forests import setup, plot, load, fit, utils, prepare

In [2]:
setup.plotting(remote=False)

In [3]:
%load_ext autoreload
%autoreload 2

Load the raw FIA data grouped by repeated inventories


In [161]:
df = load.fia(store="local", states="conus", group_repeats=True)
df = load.terraclim(
    store="local",
    tlim=(int(df["year_0"].min()), 2020),
    data_vars=["ppt", "tavg", "pdsi", "pet"],
    data_aggs=["sum", "mean", "mean", "mean"],
    df=df,
    group_repeats=True,
)

In [171]:
df = pd.read_csv(
    "/Users/freeman/Dropbox (Personal)/fia_wide_with_terraclim_v5.csv",
    low_memory=False,
)

Prepare data for model fitting


In [206]:
x, y, meta = prepare.drought(df)

In [207]:
x_z, x_mean, x_std = utils.zscore_2d(x)

Fit models


In [208]:
pf = meta.copy()
pf["observed"] = y
pf["predicted"] = np.NaN

models = {}
for code in pf["type_code"].unique():
    inds = pf["type_code"] == code
    model = fit.hurdle(x=x_z[inds], y=y[inds])
    models[code] = model
    pf.loc[inds, "predicted"] = model.predict(x=x_z[inds])

pf = pf[pf["predicted"] < 1]

Compute simple score metrics (on all data and just on the values where mortality
was positive)


In [209]:
pf[["observed", "predicted"]].corr().iloc[0, 1] ** 2

0.05638803314813913

In [210]:
pf[pf["observed"] > 0][["observed", "predicted"]].corr().iloc[0, 1] ** 2

0.0814934791894348

Plot observed vs predicted


In [211]:
plot.xy(
    x=pf["observed"],
    y=pf["predicted"],
    xlabel="measured mort ratio",
    ylabel="predicted mort ratio",
    xlim=(0, 1),
    ylim=(0, 1),
    width=300,
    height=300,
    opacity=0.1,
)

alt.Chart(...)

Make a map of actual mortality ratios


In [212]:
plot.carto(
    lat=pf["lat"],
    lon=pf["lon"],
    color=pf["observed"],
    cmap="blues",
    clim=(0, 0.1),
    size=2,
    opacity=0.75,
)

alt.LayerChart(...)

Make a map of predicted mortality ratios


In [217]:
plot.carto(
    lat=pf["lat"],
    lon=pf["lon"],
    color=pf["predicted"],
    clabel="mortality",
    cmap="blues",
    clim=(0, 0.1),
    size=2,
    opacity=0.75,
)

alt.LayerChart(...)